<a href="https://colab.research.google.com/github/Sebastiao199/Project3MRS/blob/main/list_movies_by_genre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

In [2]:
df_titlebasics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t')

# Contains the following information for titles:
# tconst (string) - alphanumeric unique identifier of the title // Interesting. Primary Key?
# titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc) // Interesting, maybe worths deleting the ones that are not the title type = "movie"
# primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release // Interesting
# originalTitle (string) - original title, in the original language // Drop (?)
# isAdult (boolean) - 0: non-adult title; 1: adult title // Interesting, but needs to be cleaned, there are some non booleans value.
# startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year // Interesting
# endYear (YYYY) – TV Series end year. ‘\N’ for all other title types // Drop (?), the ones with end year might be tvseries.
# runtimeMinutes – primary runtime of the title, in minutes // Interesting, maybe worths deleting the ones that are less than 60min taking in consideration the purpose are just movies.
# genres (string array) – includes up to three genres associated with the title // Interesting, there's interest values.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_titlebasics.head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [ ]:
df_titlebasics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9320186 entries, 0 to 9320185
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 640.0+ MB


In [ ]:
df_titlebasics.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,9320186,9320186,9320175,9320175,9320186,9320186,9320186,9320186,9320176
unique,9320186,11,4268462,4289109,11,151,96,878,2330
top,tt0000001,tvEpisode,Episode #1.1,Episode #1.1,0,\N,\N,\N,Drama
freq,1,7033369,45574,45574,8967349,1247405,9222490,6793573,1058209


In [ ]:
df_titlebasics.nunique()
#isAdult has 11 unique values, when supposely is boolean
#runtimeMinutes < 60 min might not make sense

tconst            9320186
titleType              11
primaryTitle      4268462
originalTitle     4289109
isAdult                11
startYear             151
endYear                96
runtimeMinutes        878
genres               2330
dtype: int64

In [ ]:
df_titlebasics['genres'].unique()

array(['Documentary,Short', 'Animation,Short', 'Animation,Comedy,Romance',
       ..., 'Biography,Crime,Music', 'Mystery,Reality-TV,Thriller',
       'Musical,Reality-TV,Talk-Show'], dtype=object)

In [ ]:
df_titlebasics['genres'].unique()

array(['Documentary,Short', 'Animation,Short', 'Animation,Comedy,Romance',
       ..., 'Biography,Crime,Music', 'Mystery,Reality-TV,Thriller',
       'Musical,Reality-TV,Talk-Show'], dtype=object)

In [ ]:
df_titlebasics['titleType'].isna().sum()

0

In [ ]:
# df_titlebasics.groupby('genres')['primaryTitle'].sum()

In [ ]:
df_titlebasics['titleType'].unique()

array(['short', 'movie', 'tvMiniSeries', 'tvShort', 'tvMovie', 'tvSeries',
       'tvEpisode', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [3]:
movies = df_titlebasics[df_titlebasics['titleType'] == 'movie']
movies.reset_index()
# movies.groupby('genres').originalTitle

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
2,570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
3,587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
4,610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama
...,...,...,...,...,...,...,...,...,...,...
626223,9346728,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,\N,57,Documentary
626224,9346755,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,\N,100,Documentary
626225,9346767,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,\N,\N,Comedy
626226,9346778,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N


In [ ]:
movies.info

<bound method DataFrame.info of             tconst titleType  \
8        tt0000009     movie   
498      tt0000502     movie   
570      tt0000574     movie   
587      tt0000591     movie   
610      tt0000615     movie   
...            ...       ...   
9320076  tt9916622     movie   
9320103  tt9916680     movie   
9320115  tt9916706     movie   
9320126  tt9916730     movie   
9320136  tt9916754     movie   

                                              primaryTitle  \
8                                               Miss Jerry   
498                                               Bohemios   
570                            The Story of the Kelly Gang   
587                                       The Prodigal Son   
610                                     Robbery Under Arms   
...                                                    ...   
9320076        Rodolpho Teóphilo - O Legado de um Pioneiro   
9320103  De la ilusión al desconcierto: cine colombiano...   
9320115                  

In [4]:
movies.drop(['isAdult', 'endYear', 'runtimeMinutes'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
movies.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,1894,Romance
498,tt0000502,movie,Bohemios,Bohemios,1905,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,1907,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,1907,Drama


In [ ]:
print(movies['genres'].unique())

['Romance' '\\N' 'Action,Adventure,Biography' ...
 'Music,Musical,Reality-TV' 'Action,Crime,Short' 'News,Sport,Talk-Show']


In [ ]:
print(movies['genres'].nunique())

1470


In [5]:
movies_final = movies[movies['genres'] != "\\N"]
movies_final.drop(['titleType', 'originalTitle'], axis=1, inplace=True)
movies_final.reset_index()
movies_final.head()

,tconst,primaryTitle,startYear,genres
8,tt0000009,Miss Jerry,1894,Romance
570,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,1907,Drama
610,tt0000615,Robbery Under Arms,1907,Drama
625,tt0000630,Hamlet,1908,Drama


In [ ]:
movies_final['genres'].nunique()

1469

In [6]:
genres_separated = movies_final.groupby(['genres']).nunique()[['tconst']].reset_index()
genres_separated['genres'] = genres_separated.genres.apply(lambda genre: genre.split('/') and genre.split(','))
genres_separated = genres_separated.explode('genres')
genres_separated.groupby('genres').agg({'tconst':'sum'})

,tconst
genres,
Action,52315
Adult,8819
Adventure,27335
Animation,8606
Biography,16658
Comedy,106041
Crime,35872
Documentary,117234
Drama,225199


In [7]:
condition_remove_some_genres = movies_final[((movies_final['genres'].str.contains('Film-Noir'))==False) & ((movies_final['genres'].str.contains('Game-Show'))==False) & ((movies_final['genres'].str.contains('News'))==False) & ((movies_final['genres'].str.contains('Reality-TV'))==False) & ((movies_final['genres'].str.contains('Short'))==False) & ((movies_final['genres'].str.contains('Talk-Show'))==False)]
condition_remove_some_genres

,tconst,primaryTitle,startYear,genres
8,tt0000009,Miss Jerry,1894,Romance
570,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography"
587,tt0000591,The Prodigal Son,1907,Drama
610,tt0000615,Robbery Under Arms,1907,Drama
625,tt0000630,Hamlet,1908,Drama
...,...,...,...,...
9346727,tt9916620,The Copeland Case,\N,Drama
9346728,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,Documentary
9346755,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,Documentary
9346767,tt9916706,Dankyavar Danka,2013,Comedy


In [8]:
genres_separated2 = condition_remove_some_genres.groupby(['genres']).nunique()[['tconst']].reset_index()
genres_separated2['genres'] = genres_separated2.genres.apply(lambda genre: genre.split('/') and genre.split(','))
genres_separated2 = genres_separated2.explode('genres')
genres_separated2.groupby('genres').agg({'tconst':'sum'})

,tconst
genres,
Action,52287
Adult,8815
Adventure,27272
Animation,8598
Biography,16538
Comedy,105963
Crime,35182
Documentary,115773
Drama,224282


In [9]:
def func(single_genre, movie_genre):
    return single_genre in movie_genre

no_genre = 'Adult'
movie_gende_value_yes = 'Adult,Adventure,Romance'
movie_gende_value_no = 'Action,Adventure,Romance'
func(no_genre,movie_gende_value_no)

False

In [ ]:
movies_final[movies_final['genres'] == 'News']

,tconst,primaryTitle,startYear,genres
465000,tt0483983,Entrevista Com o Surdo Gay,2004,News
740850,tt0764869,Punyal na ginto,1933,News
1022007,tt10097464,Butch Hartman's OAXIS Plan,2018,News
1022018,tt10097482,Butch Hartman's Cult of Personality: OAXIS Dis...,2018,News
1428870,tt1082847,Ja en tenim prou,2007,News
1499561,tt10954574,Margaret Atwood: Live in Cinemas,2019,News
2120088,tt12069158,KBEV/92nd Academy Awards - Full Coverage,2020,News
2381070,tt12552184,Intelligence Community Abuses: The Interviews,\N,News
2417079,tt12622258,Brexit to be or not to be,2018,News
2417410,tt12622858,The Best You Expo,2020,News


In [ ]:
condition_remove_some_genres[condition_remove_some_genres['primaryTitle'] == 'Burger King: with Michelle Trachtenberg	']

,tconst,primaryTitle,startYear,genres


In [ ]:
condition_remove_some_genres.shape

(550326, 4)

In [10]:
title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t')

In [ ]:
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
2,tt0000003,6.5,1728
3,tt0000004,5.6,173
4,tt0000005,6.2,2541


In [ ]:
title_ratings.shape

(1241487, 3)

In [11]:
movies_and_ratings = pd.merge(condition_remove_some_genres, title_ratings, on='tconst', how='left')
movies_and_ratings.head(10)

,tconst,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,Romance,5.2,200.0
1,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,794.0
2,tt0000591,The Prodigal Son,1907,Drama,5.1,20.0
3,tt0000615,Robbery Under Arms,1907,Drama,4.3,23.0
4,tt0000630,Hamlet,1908,Drama,2.9,25.0
5,tt0000675,Don Quijote,1908,Drama,4.2,19.0
6,tt0000679,The Fairylogue and Radio-Plays,1908,"Adventure,Fantasy",5.2,66.0
7,tt0000886,"Hamlet, Prince of Denmark",1910,Drama,4.7,38.0
8,tt0000941,Locura de amor,1909,Drama,4.5,23.0
9,tt0001028,Salome Mad,1909,Comedy,4.1,17.0


In [12]:
movies_and_ratings.sort_values('averageRating', ascending=False)

,tconst,primaryTitle,startYear,genres,averageRating,numVotes
523105,tt8215820,Are We Making a Film? Behind the Scenes: Follo...,2018,Documentary,10.0,8.0
297388,tt15481906,Transilvania for Sierra Leone,2021,Documentary,10.0,12.0
388484,tt2720024,Journey 4 Artists,2013,"Biography,Documentary,History",10.0,7.0
319216,tt1737585,East of Byzantium: War Gods and Warrior Saints,2016,Documentary,10.0,9.0
319149,tt17371496,Meeting Mr. Christmas,2022,Romance,10.0,6.0
...,...,...,...,...,...,...
551201,tt9916620,The Copeland Case,\N,Drama,NaN,NaN
551202,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,Documentary,NaN,NaN
551203,tt9916680,De la ilusión al desconcierto: cine colombiano...,2007,Documentary,NaN,NaN
551204,tt9916706,Dankyavar Danka,2013,Comedy,NaN,NaN


In [ ]:
movies_and_ratings['averageRating'].isna().sum()

279883

In [13]:
movies_and_ratings.dropna(subset=['averageRating'], inplace = True)

In [ ]:
movies_and_ratings['averageRating'].isna().sum()

0

In [ ]:
movies_and_ratings['numVotes'].isna().sum()

0

In [14]:
movies_and_ratings

,tconst,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,1894,Romance,5.2,200.0
1,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",6.0,794.0
2,tt0000591,The Prodigal Son,1907,Drama,5.1,20.0
3,tt0000615,Robbery Under Arms,1907,Drama,4.3,23.0
4,tt0000630,Hamlet,1908,Drama,2.9,25.0
...,...,...,...,...,...,...
551196,tt9916190,Safeguard,2020,"Action,Adventure,Thriller",3.6,238.0
551197,tt9916270,Il talento del calabrone,2020,Thriller,5.8,1370.0
551198,tt9916362,Coven,2020,"Drama,History",6.4,4959.0
551199,tt9916428,The Secret of China,2019,"Adventure,History,War",3.8,14.0


In [15]:
movies_and_ratings_five_percent = movies_and_ratings[movies_and_ratings['numVotes'] > 6096]

In [16]:
movies_and_ratings_five_percent.head()

,tconst,primaryTitle,startYear,genres,averageRating,numVotes
626,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0
1527,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0
3538,tt0009968,Broken Blossoms,1919,"Drama,Romance",7.3,10442.0
3784,tt0010323,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",8.0,64403.0
4413,tt0011237,The Golem,1920,"Fantasy,Horror",7.2,7834.0


In [ ]:
movies_and_ratings_five_percent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13560 entries, 626 to 550867
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         13560 non-null  object 
 1   primaryTitle   13560 non-null  object 
 2   startYear      13560 non-null  object 
 3   genres         13560 non-null  object 
 4   averageRating  13560 non-null  float64
 5   numVotes       13560 non-null  float64
dtypes: float64(2), object(4)
memory usage: 741.6+ KB


In [50]:
rotten_tomatoes_rating = pd.read_csv('https://raw.githubusercontent.com/Sebastiao199/Project3MRS/main/Rating%20RT.csv')
rotten_tomatoes_rating.drop(['Unnamed: 0'], axis=1, inplace=True)
rotten_tomatoes_rating

,tconst,tomatometer_rating
0,tt0343172,68.0
1,tt0031322,83.0
2,tt2316868,45.0
3,tt1166100,50.0
4,tt0449951,50.0
...,...,...
11829,tt0488604,17.0
11830,tt2404738,86.0
11831,tt1572491,77.0
11832,tt0330602,32.0


In [51]:
imdb_rotten_merge = pd.merge(movies_and_ratings_five_percent, rotten_tomatoes_rating, on='tconst', how='left')
imdb_rotten_merge

,tconst,primaryTitle,startYear,genres,averageRating,numVotes,tomatometer_rating
0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0
1,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0,97.0
2,tt0009968,Broken Blossoms,1919,"Drama,Romance",7.3,10442.0,95.0
3,tt0010323,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",8.0,64403.0,NaN
4,tt0011237,The Golem,1920,"Fantasy,Horror",7.2,7834.0,NaN
...,...,...,...,...,...,...,...
13564,tt9893250,I Care a Lot,2020,"Comedy,Crime,Drama",6.3,132724.0,NaN
13565,tt9894470,VFW,2019,"Action,Crime,Horror",6.1,8565.0,81.0
13566,tt9898858,Coffee & Kareem,2020,"Action,Comedy,Crime",5.1,13807.0,20.0
13567,tt9900782,Kaithi,2019,"Action,Adventure,Crime",8.5,31848.0,NaN


In [ ]:
imdb_rotten_merge['tomatometer_rating'].isna().sum()

6612

In [ ]:
type(imdb_rotten_merge['genres'][0])

str

In [ ]:
imdb_rotten_merge['genres'] = imdb_rotten_merge['genres'].tolist()

In [ ]:
type(imdb_rotten_merge['genres'][0])

str

In [52]:
imdb_rotten_merge['genresfinal'] = imdb_rotten_merge['genres'].apply(lambda x: x.split(','))
final_explosion_version = imdb_rotten_merge.explode('genresfinal').reset_index()
# movies_final.drop(['titleType', 'originalTitle'], axis=1, inplace=True)

In [53]:
final_explosion_version['startYear'] = final_explosion_version['startYear'].astype(str)
final_explosion_version['startYear'] = final_explosion_version['startYear'].apply(lambda x : x[0:4])
final_explosion_version['startYear'] = final_explosion_version['startYear'].apply(lambda x : int(x))
final_explosion_version['startYear']

0        1915
1        1915
2        1915
3        1916
4        1916
         ... 
34292    2019
34293    2019
34294    2021
34295    2021
34296    2021
Name: startYear, Length: 34297, dtype: int64

In [38]:
final_explosion_version

,index,tconst,primaryTitle,startYear,genres,averageRating,numVotes,tomatometer_rating,genresfinal
0,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,Drama
1,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,History
2,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,War
3,1,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0,97.0,Drama
4,1,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0,97.0,History
...,...,...,...,...,...,...,...,...,...
34292,13567,tt9900782,Kaithi,2019,"Action,Adventure,Crime",8.5,31848.0,NaN,Adventure
34293,13567,tt9900782,Kaithi,2019,"Action,Adventure,Crime",8.5,31848.0,NaN,Crime
34294,13568,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",6.2,11143.0,NaN,Fantasy
34295,13568,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",6.2,11143.0,NaN,Horror


In [54]:
final_explosion_version.rename(columns = {'primaryTitle': 'Movie title', 
                                    'startYear': 'Year',
                                    'averageRating': 'IMDb rating',
                                    'tomatometer_rating': 'Rotten Tomatoes rating',
                                    'genresfinal': 'Genre',
                                    'genres': 'Genres',
                                    'numVotes': 'Nb votes'
                                    },
                         inplace = True)

# Director name 
# Movie title 
# Year
# IMDb rating

In [ ]:
# final_table = imdb_rotten_merge[imdb_rotten_merge['genres'].apply(lambda genre: genre.split(',')).explode()]

In [55]:
final_explosion_version

,index,tconst,Movie title,Year,Genres,IMDb rating,Nb votes,Rotten Tomatoes rating,Genre
0,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,Drama
1,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,History
2,0,tt0004972,The Birth of a Nation,1915,"Drama,History,War",6.2,24930.0,93.0,War
3,1,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0,97.0,Drama
4,1,tt0006864,Intolerance,1916,"Drama,History",7.7,15699.0,97.0,History
...,...,...,...,...,...,...,...,...,...
34292,13567,tt9900782,Kaithi,2019,"Action,Adventure,Crime",8.5,31848.0,NaN,Adventure
34293,13567,tt9900782,Kaithi,2019,"Action,Adventure,Crime",8.5,31848.0,NaN,Crime
34294,13568,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",6.2,11143.0,NaN,Fantasy
34295,13568,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",6.2,11143.0,NaN,Horror


In [56]:
final_explosion_version.sort_values(by=["IMDb rating"], ascending=False).head(50)

,index,tconst,Movie title,Year,Genres,IMDb rating,Nb votes,Rotten Tomatoes rating,Genre
24429,9667,tt16747572,The Silence of Swastika,2021,"Documentary,History",9.7,10141.0,NaN,Documentary
24430,9667,tt16747572,The Silence of Swastika,2021,"Documentary,History",9.7,10141.0,NaN,History
8697,3524,tt0111161,The Shawshank Redemption,1994,Drama,9.3,2656994.0,91.0,Drama
2954,1226,tt0068646,The Godfather,1972,"Crime,Drama",9.2,1841519.0,98.0,Drama
2953,1226,tt0068646,The Godfather,1972,"Crime,Drama",9.2,1841519.0,98.0,Crime
12707,5081,tt0252487,The Chaos Class,1975,"Comedy,Drama",9.2,40813.0,NaN,Drama
12706,5081,tt0252487,The Chaos Class,1975,"Comedy,Drama",9.2,40813.0,NaN,Comedy
12880,5152,tt0259534,Ramayana: The Legend of Prince Rama,1993,"Action,Adventure,Animation",9.2,7624.0,NaN,Animation
12879,5152,tt0259534,Ramayana: The Legend of Prince Rama,1993,"Action,Adventure,Animation",9.2,7624.0,NaN,Adventure
12878,5152,tt0259534,Ramayana: The Legend of Prince Rama,1993,"Action,Adventure,Animation",9.2,7624.0,NaN,Action


In [57]:
final_explosion_version['Rotten Tomatoes rating'] = final_explosion_version['Rotten Tomatoes rating']/10
final_explosion_version['Rotten Tomatoes rating'].head(20)

0     9.3
1     9.3
2     9.3
3     9.7
4     9.7
5     9.5
6     9.5
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
Name: Rotten Tomatoes rating, dtype: float64

In [59]:
final_explosion_version.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34297 entries, 0 to 34296
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   34297 non-null  int64  
 1   tconst                  34297 non-null  object 
 2   Movie title             34297 non-null  object 
 3   Year                    34297 non-null  int64  
 4   Genres                  34297 non-null  object 
 5   IMDb rating             34297 non-null  float64
 6   Nb votes                34297 non-null  float64
 7   Rotten Tomatoes rating  17844 non-null  float64
 8   Genre                   34297 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.4+ MB


In [58]:
final_explosion_version.to_csv('GenreQuestion.csv')

In [ ]:
francisco_table = pd.read_csv('https://raw.githubusercontent.com/Sebastiao199/Project3MRS/main/tprn_10.csv')
francisco_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68498 entries, 0 to 68497
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   68498 non-null  int64  
 1   tconst       68498 non-null  object 
 2   Movie title  68498 non-null  object 
 3   Year         68498 non-null  int64  
 4   Genres       68498 non-null  object 
 5   nconst       68491 non-null  object 
 6   category     68491 non-null  object 
 7   IMDb rating  68498 non-null  float64
 8   Nb votes     68498 non-null  float64
 9   Staff name   68485 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 5.2+ MB


In [ ]:
francisco_table['Movie title'].nunique()

13016